In [ ]:

import sys
reload(sys)
sys.setdefaultencoding("utf-8")
sys.path.append('../lib/')
import ImportNotebook
%matplotlib inline

import numpy as np
import numpy.matrixlib
import cv2
import time
import asmgazetracker
import matplotlib.pyplot as plt

import DaugmanIrisLocator as pcl

In [ ]:
t = time.time()
print '开始生成模板时间=%f\n'%(t)
pcl.generateTemplateSet()
t = time.time()
print '模板完成时间=%f\n'%(t)
pcl.irisRange

In [ ]:
userProfilePath = '/Users/ly/eyeProject/current_head/test/user_profile/liuyang/trackermodel.yaml'
# backgroundUserProfilePath = '/Users/Jiangzp/trackingdata/user_profile/jzp214/trackermodel.yaml'
cameraProfilePath = '/Users/ly/eyeProject/current_head/test/camera_calibration/iSight/CM_iSight.yaml'

In [ ]:
pt = asmgazetracker.ASM_Gaze_Tracker()
pt.initialize(userProfilePath,cameraProfilePath)
# pt.feedBackgroundTracker(backgroundUserProfilePath)
pt.isImageRectificationUsingHomoTransformation = 0
pt.poseEstimationUsingRansac = 0
pt.poseEstimationRobustTracker = 0

In [ ]:
leyefinalList = [];
reyefinalList = [];
lIrisRadii = [];
rIrisRadii = [];
plt.ion()

# leftDiffImage = plt.figure('leftDiffImage')
# leftDiffImage.add_subplot(1,1,1).imshow(np.zeros((5,5)))
# leftDiffImage.canvas.draw()

# rightDiffImage = plt.figure('rightDiffImage')
# rightDiffImage.add_subplot(1,1,1).imshow(np.zeros((5,5)))
# rightDiffImage.canvas.draw()

allTrackedPointsList = [];

t = time.time()
print '打开摄像头时间=%f\n'%(t)
# cv2.startWindowThread()
cap = cv2.VideoCapture(0)
frameCount = 0
#pcl.generateTemplateSet()
#while(True):
while(frameCount < 21):
    
    print '-' *50
    print "count=%d\n"%frameCount
    t = time.time()
    print t
    frameCount +=1
    #读取视频帧
    ret, frame = cap.read()
#     frame = cv2.medianBlur(frame,3)
    #基于特征跟踪类？什么原理
    succecced = pt.featureTracking(frame)
    if succecced:
        pt.estimateFacePose()
        pt.extractEyeRectifiedImages()
#        cv2.imshow('left',pt.rightEyeImageRectified)
#        cv2.imshow('right',pt.leftEyeImageRectified)
        #python的RGB和opencv的RGB通道是反的
        leftEyeRectified = cv2.cvtColor(pt.leftEyeImageRectified,cv2.COLOR_BGR2RGB)
        rightEyeRectified = cv2.cvtColor(pt.rightEyeImageRectified,cv2.COLOR_BGR2RGB)
#        cv2.imshow('test',rightEyeRectified)
        rectifiedImage = cv2.cvtColor(pt.rectifiedImage,cv2.COLOR_BGR2RGB)
        #
        if frameCount == 20:
            pcl.regulateIrisRange()
            print 'regulated'
            print pcl.irisRange
        
        #eyecenterInRect是在rect中黑眼仁的圆心
        #IrisRadius是黑眼仁的半径
        #先处理左眼，后处理右眼
        leyecenterInRect,lIrisRadius,diffLeft,leftDaugmanCenter,leftDaugmanRadius,leftEyeFiltered = pcl.DaugmanIrisLocalization(leftEyeRectified)
        reyecenterInRect,rIrisRadius,diffRight,rightDaugmanCenter,rightDaugmanRadius,rightEyeFiltered = pcl.DaugmanIrisLocalization(rightEyeRectified)

#        if frameCount%2 == 0:
#            print '偶数，处理左眼，右眼用左眼的模板'
#            leyecenterInRect,lIrisRadius,diffLeft,leftDaugmanCenter,leftDaugmanRadius,leftEyeFiltered = pcl.DaugmanIrisLocalization(leftEyeRectified)
#            reyecenterInRect,rIrisRadius,diffRight,rightDaugmanCenter,rightDaugmanRadius,rightEyeFiltered = pcl.DaugmanIrisLocalizationAnothor(rightEyeRectified)
#        else:
#            print '奇数，处理右眼，左眼用右眼的模板'
#            reyecenterInRect,rIrisRadius,diffRight,rightDaugmanCenter,rightDaugmanRadius,rightEyeFiltered = pcl.DaugmanIrisLocalization(rightEyeRectified)
#            leyecenterInRect,lIrisRadius,diffLeft,leftDaugmanCenter,leftDaugmanRadius,leftEyeFiltered = pcl.DaugmanIrisLocalizationAnothor(leftEyeRectified)

        
        
        #         diffRight /= (diffRight.max()/255.0)
#         diffRight = cv2.applyColorMap(diffRight,cv2.COLORMAP_JET)
#         cv2.imshow('diffRight',diffRight)
#         leftDiffImage.axes[0].clear()
#         leftDiffImage.axes[0].imshow(diffLeft)
#         leftDiffImage.canvas.draw()
        
#         rightDiffImage.axes[0].clear()
#         rightDiffImage.axes[0].imshow(diffRight)
#         rightDiffImage.canvas.draw()
        
#         leftEyeFiltered = cv2.cvtColor(leftEyeFiltered,cv2.COLOR_RGB2BGR)
#         rightEyeFiltered = cv2.cvtColor(rightEyeFiltered,cv2.COLOR_RGB2BGR)
        lIrisRadii.append(lIrisRadius/leftEyeRectified.shape[1])
        rIrisRadii.append(rIrisRadius/leftEyeRectified.shape[1])
#         cv2.circle(leftEyeFiltered,tuple(np.int32(np.squeeze(leftDaugmanCenter))),int(leftDaugmanRadius),(0,0,255))
#         cv2.circle(rightEyeFiltered,tuple(np.int32(np.squeeze(rightDaugmanCenter))),int(rightDaugmanRadius),(0,0,255))
#         cv2.imshow('lefteye',leftEyeFiltered)
#         cv2.imshow('righteye',rightEyeFiltered)
        
        leyecenterInRectified = leyecenterInRect + pt.leftEyeRectTlPoint.transpose()
        reyecenterInRectified = reyecenterInRect + pt.rightEyeRectTlPoint.transpose()
        
        if pt.isImageRectificationUsingHomoTransformation:
            #透视变换，为了？
            leyecenterfinal = cv2.perspectiveTransform(leyecenterInRectified[None,:,:],pt.rectifyTransformationInv)
            reyecenterfinal = cv2.perspectiveTransform(reyecenterInRectified[None,:,:],pt.rectifyTransformationInv)
        else:
            leyecenterInRectified[0][0] = frame.shape[1] - leyecenterInRectified[0][0]
            reyecenterInRectified[0][0] = frame.shape[1] - reyecenterInRectified[0][0]
            leyecenterfinal = cv2.transform(leyecenterInRectified[None,:,:],pt.rectifyTransformationInv)
            reyecenterfinal = cv2.transform(reyecenterInRectified[None,:,:],pt.rectifyTransformationInv)            
            
        leyefinalList.append(leyecenterfinal)
        reyefinalList.append(reyecenterfinal)
        allTrackedPointsList.append(pt.allTrackedPoints2D)
        #cv2.circle(图像，圆心，半径，颜色)
        for p in pt.allTrackedPoints2D:
            cv2.circle(frame,tuple(p),1,(255,255,255))
        for p in pt.selectedTrackedPoints2D:
            cv2.circle(frame,tuple(p),1,(0,255,0))
        #print '-' *40
        #print 'leyecenterfinal'
        #print leyecenterfinal
        #print 'tuple(np.int32(np.squeeze(leyecenterfinal)))'
        #print leyecenterfinal
        #print np.squeeze(leyecenterfinal)
        #print np.int32(np.squeeze(leyecenterfinal))
        #print tuple(np.int32(np.squeeze(leyecenterfinal)))
        cv2.circle(frame,tuple(np.int32(np.squeeze(leyecenterfinal))),1,(0,0,255))
        cv2.circle(frame,tuple(np.int32(np.squeeze(leyecenterfinal))),np.int32(lIrisRadius),(0,0,255))
        cv2.circle(frame,tuple(np.int32(np.squeeze(reyecenterfinal))),1,(0,0,255))
        cv2.circle(frame,tuple(np.int32(np.squeeze(reyecenterfinal))),np.int32(rIrisRadius),(0,0,255))
    cv2.imshow('frame',frame)
    
    keyValue = cv2.waitKey(1) & 0xFF 
    if keyValue == ord('q'):
        del(cap)
        cv2.destroyAllWindows()
        break
    elif keyValue == ord('d'):
        pt.reDetectFace()


In [ ]:
# %matplotlib inline
leyefinalArray = np.concatenate(leyefinalList).squeeze()
reyefinalArray = np.concatenate(reyefinalList).squeeze()
allTrackedPointsArray = np.concatenate(allTrackedPointsList,2)
plt.figure(0)
for i in range(1):
    plt.scatter(allTrackedPointsArray[i,0,:],allTrackedPointsArray[i,1,:],5,range(0,allTrackedPointsArray.shape[2]))
# plt.figure(1)
# plt.scatter(leyefinalArray[:,0],leyefinalArray[:,1],20,range(0,leyefinalArray.shape[0]))
# plt.scatter(reyefinalArray[:,0],reyefinalArray[:,1],20,range(0,reyefinalArray.shape[0]))
# plt.show()

In [ ]:
cIL = pt.stringIndexToIntIndex('canthusInnerLeft')
cIR = pt.stringIndexToIntIndex('canthusInnerRight')
cOL = pt.stringIndexToIntIndex('canthusOuterLeft')
cOR = pt.stringIndexToIntIndex('canthusOuterRight')
cILArray = allTrackedPointsArray[cIL,:,:]
cIRArray = allTrackedPointsArray[cIR,:,:]
cOLArray = allTrackedPointsArray[cOL,:,:]
cORArray = allTrackedPointsArray[cOR,:,:]
mid = (cILArray + cIRArray + cOLArray + cORArray)/4
leyeWRT = leyefinalArray - mid.transpose()
reyeWRT = reyefinalArray - mid.transpose()
plt.figure('wrt')
plt.scatter(leyeWRT[:,0],leyeWRT[:,1],20,range(0,leyeWRT.shape[0]))
plt.figure('mid')
plt.scatter(leyeWRT[:,0],leyeWRT[:,1],20,range(0,leyeWRT.shape[0]))
plt.scatter(mid[:,0],mid[:,1],20,range(0,mid.shape[0]))
plt.show('wrt')
plt.show('mid')


In [ ]:
pcl.irisRange